In [185]:
from elasticsearch import Elasticsearch
import tensorflow as tf
from tensorflow import keras
import keras
import re
import multiprocessing
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import string
from gensim.models import word2vec
from keras import losses
from keras.models import Sequential
from keras.layers import Dense, Softmax
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import asarray
import nltk
nltk.download('stopwords')
from numpy import zeros
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer # Used for stemming

[nltk_data] Downloading package punkt to /home/spetz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/spetz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [186]:
csv  = pd.read_csv("final_cluster.csv")
csv = csv[['cluster' , 'uid' ,'location','age' , 'isbn' ,'book_title' ,'book_author','summary','rating']]

csv

,cluster,uid,location,age,isbn,book_title,book_author,summary,rating
0,0,19,"weston, ,",14.0,0375759778,Prague : A Novel,ARTHUR PHILLIPS,Five American expatriates living in Budapest i...,7
1,0,51,"renton, washington, usa",34.0,0440225701,The Street Lawyer,JOHN GRISHAM,"After a violent encounter with a homeless man,...",9
2,0,75,"long beach, california, usa",37.0,1558531025,Life's Little Instruction Book (Life's Little ...,H. Jackson Brown,A collection of advice on how to live a happy ...,5
3,0,125,"lansing, michigan, usa",49.0,0679429220,Midnight in the Garden of Good and Evil: A Sav...,John Berendt,"In charming, beautiful, and wealthy old-South ...",10
4,0,178,"rexford, new york, usa",28.0,0451410319,Hush,Anne Frasier,Sixteen years after her own son fell victim to...,6
...,...,...,...,...,...,...,...,...,...
22575,1,278542,"greeneville, tennessee, usa",39.0,0446605409,Plum Island,Nelson DeMille,"Fast-paced and atmospheric, marked by entranci...",0
22576,1,278692,"butler, western australia, australia",26.0,0060976845,Little Altars Everywhere: A Novel,Rebecca Wells,"Don&#39;t miss Little Altars Everywhere, the N...",3
22577,1,278849,"georgetown, ontario, canada",23.0,0445210214,Mythology 101 (Questar Fantasy),Jody Lynn Nye,Keith Doyle is determined in his campaign to t...,0
22578,1,278849,"georgetown, ontario, canada",23.0,0771012837,"Klondike: The Last Great Gold Rush, 1896-1899",Pierre Berton,The Klondike stampede was a wild interlude in ...,0


In [187]:
# Delete all the rows from dataframe which have rating = 0
cluster = csv.where(csv['rating'] != 0).dropna()
#cluster1_new = csv_cluster1.where(csv_cluster1['rating'] != 0).dropna()
cluster

,cluster,uid,location,age,isbn,book_title,book_author,summary,rating
0,0.0,19.0,"weston, ,",14.0,0375759778,Prague : A Novel,ARTHUR PHILLIPS,Five American expatriates living in Budapest i...,7.0
1,0.0,51.0,"renton, washington, usa",34.0,0440225701,The Street Lawyer,JOHN GRISHAM,"After a violent encounter with a homeless man,...",9.0
2,0.0,75.0,"long beach, california, usa",37.0,1558531025,Life's Little Instruction Book (Life's Little ...,H. Jackson Brown,A collection of advice on how to live a happy ...,5.0
3,0.0,125.0,"lansing, michigan, usa",49.0,0679429220,Midnight in the Garden of Good and Evil: A Sav...,John Berendt,"In charming, beautiful, and wealthy old-South ...",10.0
4,0.0,178.0,"rexford, new york, usa",28.0,0451410319,Hush,Anne Frasier,Sixteen years after her own son fell victim to...,6.0
...,...,...,...,...,...,...,...,...,...
22568,1.0,278096.0,"belle isle, florida, usa",57.0,0440236053,Writ of Execution,Perri O'Shaughnessy,Nina Reilly takes on the case of Jessie Potter...,2.0
22569,1.0,278212.0,"chicago, illinois, usa",33.0,0688170528,The Pact: A Love Story,Jodi Picoult,It&#39;s rare to find a writer who combines Al...,5.0
22571,1.0,278373.0,"dubbo, new south wales, australia",13.0,0971880107,Wild Animus,Rich Shapero,"Wild animus is a search for the primordial, a ...",1.0
22572,1.0,278431.0,"alexandria, virginia, usa",39.0,0446608262,The Lion's Game,Nelson DeMille,Filled with unrelenting suspense and surprisin...,2.0


In [188]:
avg_rating = cluster.groupby(['isbn'])['rating'].mean().reset_index(name='rating')
#avg_rating1 = cluster1_new.groupby(['isbn'])['rating'].mean().reset_index(name='rating')

avg_rating

,isbn,rating
0,0002005395,8.0
1,0002214067,10.0
2,0002237857,4.0
3,0002251760,9.0
4,0002253402,6.0
...,...,...
8930,9171492674,10.0
8931,9507399097,10.0
8932,9628606727,10.0
8933,9722107909,8.0


In [189]:
# Create user rating dataframe ,whice contains all info about each movie rating
new_df = cluster[['cluster','summary','uid','rating','isbn']]
new_df

,cluster,summary,uid,rating,isbn
0,0.0,Five American expatriates living in Budapest i...,19.0,7.0,0375759778
1,0.0,"After a violent encounter with a homeless man,...",51.0,9.0,0440225701
2,0.0,A collection of advice on how to live a happy ...,75.0,5.0,1558531025
3,0.0,"In charming, beautiful, and wealthy old-South ...",125.0,10.0,0679429220
4,0.0,Sixteen years after her own son fell victim to...,178.0,6.0,0451410319
...,...,...,...,...,...
22568,1.0,Nina Reilly takes on the case of Jessie Potter...,278096.0,2.0,0440236053
22569,1.0,It&#39;s rare to find a writer who combines Al...,278212.0,5.0,0688170528
22571,1.0,"Wild animus is a search for the primordial, a ...",278373.0,1.0,0971880107
22572,1.0,Filled with unrelenting suspense and surprisin...,278431.0,2.0,0446608262


In [190]:
# Text-cleaning function
def clean_data(csv):
    #remove with regex all punctuation
    text = re.sub('[^A-Za-z0-9]+', ' ', csv)
    #lowercase
    text = text.lower()
    return text


new_df['Clean'] = new_df['summary'].apply(clean_data)

/tmp/ipykernel_2743/2028500723.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Clean'] = new_df['summary'].apply(clean_data)


In [191]:
stop_words = set(stopwords.words('english'))
new_df['removestopwords'] = new_df['Clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

new_df

/tmp/ipykernel_2743/810779629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['removestopwords'] = new_df['Clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


,cluster,summary,uid,rating,isbn,Clean,removestopwords
0,0.0,Five American expatriates living in Budapest i...,19.0,7.0,0375759778,five american expatriates living in budapest i...,five american expatriates living budapest earl...
1,0.0,"After a violent encounter with a homeless man,...",51.0,9.0,0440225701,after a violent encounter with a homeless man ...,violent encounter homeless man talented corpor...
2,0.0,A collection of advice on how to live a happy ...,75.0,5.0,1558531025,a collection of advice on how to live a happy ...,collection advice live happy rewarding life
3,0.0,"In charming, beautiful, and wealthy old-South ...",125.0,10.0,0679429220,in charming beautiful and wealthy old south sa...,charming beautiful wealthy old south savannah ...
4,0.0,Sixteen years after her own son fell victim to...,178.0,6.0,0451410319,sixteen years after her own son fell victim to...,sixteen years son fell victim serial killer cr...
...,...,...,...,...,...,...,...
22568,1.0,Nina Reilly takes on the case of Jessie Potter...,278096.0,2.0,0440236053,nina reilly takes on the case of jessie potter...,nina reilly takes case jessie potter desperate...
22569,1.0,It&#39;s rare to find a writer who combines Al...,278212.0,5.0,0688170528,it 39 s rare to find a writer who combines ali...,39 rare find writer combines alice hoffman 39 ...
22571,1.0,"Wild animus is a search for the primordial, a ...",278373.0,1.0,0971880107,wild animus is a search for the primordial a t...,wild animus search primordial test human found...
22572,1.0,Filled with unrelenting suspense and surprisin...,278431.0,2.0,0446608262,filled with unrelenting suspense and surprisin...,filled unrelenting suspense surprising plot tw...


In [192]:
#SnowballStemer(trasnform the word to shorter word with the same root meaning)
stemmer = SnowballStemmer("english")

#create new series('stemmed')
new_df['Stemmed'] = new_df['removestopwords'].apply(lambda x: ' '.join([stemmer.stem(word) for word in str(x).split()]))
new_df =new_df.drop(columns=['Clean'])
new_df =new_df.drop(columns=['removestopwords'])

new_df

/tmp/ipykernel_2743/611209206.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Stemmed'] = new_df['removestopwords'].apply(lambda x: ' '.join([stemmer.stem(word) for word in str(x).split()]))


,cluster,summary,uid,rating,isbn,Stemmed
0,0.0,Five American expatriates living in Budapest i...,19.0,7.0,0375759778,five american expatri live budapest earli 1990...
1,0.0,"After a violent encounter with a homeless man,...",51.0,9.0,0440225701,violent encount homeless man talent corpor law...
2,0.0,A collection of advice on how to live a happy ...,75.0,5.0,1558531025,collect advic live happi reward life
3,0.0,"In charming, beautiful, and wealthy old-South ...",125.0,10.0,0679429220,charm beauti wealthi old south savannah georgi...
4,0.0,Sixteen years after her own son fell victim to...,178.0,6.0,0451410319,sixteen year son fell victim serial killer cri...
...,...,...,...,...,...,...
22568,1.0,Nina Reilly takes on the case of Jessie Potter...,278096.0,2.0,0440236053,nina reilli take case jessi potter desper youn...
22569,1.0,It&#39;s rare to find a writer who combines Al...,278212.0,5.0,0688170528,39 rare find writer combin alic hoffman 39 gif...
22571,1.0,"Wild animus is a search for the primordial, a ...",278373.0,1.0,0971880107,wild animus search primordi test human foundat...
22572,1.0,Filled with unrelenting suspense and surprisin...,278431.0,2.0,0446608262,fill unrel suspens surpris plot twist everi te...


In [193]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
final_summaries = new_df['Stemmed'].copy()
# Tokenize the summary texts
# weight matrix of one embedding for each unique word in summary 
summaries = new_df["Stemmed"].copy()

# Tokenize the summary texts
token = Tokenizer()
token.fit_on_texts(summaries)
#len of each unique word in summary 
vocab_size = len(token.word_index) + 1 
texts = token.texts_to_sequences(summaries) # encode to integer each word 
texts = pad_sequences(texts, padding='post')
texts

array([[ 181,   35, 3594, ...,    0,    0,    0],
       [ 616,  139, 1183, ...,    0,    0,    0],
       [  68,  833,   19, ...,    0,    0,    0],
       ...,
       [  40,   61,   22, ...,    0,    0,    0],
       [ 375, 3155,  224, ...,    0,    0,    0],
       [   1,  191,  251, ...,    0,    0,    0]], dtype=int32)

In [194]:
#sequence for each word
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 400000 word vectors.


In [195]:
# Create a weight matrix of one embedding for each unique word in summary texts
embedding_matrix = zeros((vocab_size, 100))
for word, i in token.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_matrix
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.1383    ,  0.30316001, -0.11122   , ...,  0.10495   ,
         0.11652   , -0.43040001],
       [-0.26287001,  0.29894   , -0.77744001, ...,  0.20399   ,
        -0.68527001,  0.22225   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [196]:
textTrain, textTest, ratingTrain, ratingTest = train_test_split(texts, new_df['rating'], test_size=0.33)
ratingTest

21185     6.0
16803     9.0
12274     4.0
1667      6.0
16833     8.0
         ... 
11308     5.0
19599     2.0
933       7.0
4502     10.0
14503     7.0
Name: rating, Length: 5746, dtype: float64

In [197]:
from tensorflow import keras
from tensorflow.keras.losses import SparseCategoricalCrossentropy # Loss function being used
from sklearn.model_selection import train_test_split # Train-test split
import tensorflow as tf
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=texts.shape[1], trainable=False))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu' ))
model.add(keras.layers.Dense(128, activation='relu' ))
model.add(keras.layers.Dense(11, activation='softmax'))

model.summary()

# In[19]:


# Compile the model
model.compile(loss=SparseCategoricalCrossentropy(from_logits = True), optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(textTrain, ratingTrain, epochs=100, batch_size=50, validation_split = 0.2,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)])



Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 43, 100)           1648400   
                                                                 
 flatten_19 (Flatten)        (None, 4300)              0         
                                                                 
 dense_56 (Dense)            (None, 512)               2202112   
                                                                 
 dense_57 (Dense)            (None, 256)               131328    
                                                                 
 dense_58 (Dense)            (None, 128)               32896     
                                                                 
 dense_59 (Dense)            (None, 11)                1419      
                                                                 
Total params: 4,016,155
Trainable params: 2,367,755
N

/home/spetz/.local/lib/python3.10/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]
/home/spetz/.local/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


187/187 [==============================] - 2s 12ms/step - loss: 2.1312 - accuracy: 0.2173 - val_loss: 2.0982 - val_accuracy: 0.2195
Epoch 2/100
187/187 [==============================] - 2s 11ms/step - loss: 1.9280 - accuracy: 0.2875 - val_loss: 2.1443 - val_accuracy: 0.2336
Epoch 3/100
187/187 [==============================] - 2s 11ms/step - loss: 1.6403 - accuracy: 0.4142 - val_loss: 2.2376 - val_accuracy: 0.2079
Epoch 4/100
187/187 [==============================] - 2s 11ms/step - loss: 1.3147 - accuracy: 0.5383 - val_loss: 2.5616 - val_accuracy: 0.1865
Epoch 5/100
187/187 [==============================] - 2s 11ms/step - loss: 1.0880 - accuracy: 0.6044 - val_loss: 2.6843 - val_accuracy: 0.2040
Epoch 6/100
187/187 [==============================] - 2s 11ms/step - loss: 0.9460 - accuracy: 0.6206 - val_loss: 2.9509 - val_accuracy: 0.2087
Epoch 7/100
187/187 [==============================] - 2s 11ms/step - loss: 0.8577 - accuracy: 0.6281 - val_loss: 3.0843 - val_accuracy: 0.1942
Epoc

In [198]:
loss, accuracy = model.evaluate(textTest, ratingTest) # Get the loss and accuracy based on the tests



180/180 [==============================] - 0s 884us/step - loss: 4.8563 - accuracy: 0.2057


In [199]:
#cluster0-cluster1
cluster0 = csv[csv['cluster'] ==0]
cluster1=csv[csv['cluster'] == 1]

In [200]:
cluster0_null = cluster0[cluster0['rating'] == 0]

In [201]:
cluster0_null
cluster0_null = cluster0_null[['summary' ,'rating']]
cluster0_null
x=texts[0]
y = cluster0_null['rating']

cluster0_null


,summary,rating
7620,Provides an introduction to classical myths pl...,0
7622,With accurate descriptions of the changes that...,0
7626,A horse in nineteenth-century England recounts...,0
7627,By taking Gabe under his wing and teaching him...,0
7628,As a series of grisly murders terrorizes the s...,0
...,...,...
12596,Success in this area leads Sheridan to have Cu...,0
12597,Presents a fictional portrait of the relations...,0
12599,Good Faith captures the seductions and illusio...,0
12603,A novel about the difference that one person c...,0


In [204]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenize the summary texts
# weight matrix of one embedding for each unique word in summary 
summaries = cluster0_null["summary"].copy()

# Tokenize the summary texts
token = Tokenizer()
token.fit_on_texts(summaries)
#len of each unique word in summary 
vocab_size = len(token.word_index) + 1 
texts = token.texts_to_sequences(summaries) # encode to integer each word 
texts = pad_sequences(texts, padding='post')
len(texts)

2817

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().